# SCRIPT DE TEST SELENIUM AVEC NAVIGATEUR 

In [2]:
pip install selenium chromedriver-autoinstaller

  Using cached selenium-4.38.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached chromedriver_autoinstaller-0.6.4-py3-none-any.whl.metadata (2.1 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.32.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.38.0-py3-none-any.whl (9.7 MB)
Using cached chromedriver_autoinstaller-0.6.4-py3-none-any.whl (7.6 kB)
Using cached certifi-2025.10.5-py3-none-any.whl (163 kB)
Using cached trio-0.32.0-py3-none-any.w

In [3]:
import chromedriver_autoinstaller
path = chromedriver_autoinstaller.install()
print("Driver installé :", path)

CHROME >= 115, using mac-x64 as architecture identifier
Driver installé : /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/chromedriver_autoinstaller/142/chromedriver


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from selenium.common.exceptions import ElementClickInterceptedException
import time, os, csv

# installer le bon ChromeDriver selon la version de Chrome
driver_path = chromedriver_autoinstaller.install()

options = webdriver.ChromeOptions()
# Pour VOIR le navigateur, ne pas mettre --headless
# options.add_argument("--headless=new")  
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 15)


## Test sur une boucle entière avec gestion des cookies

Fonction pour cliquer sur le bouton "Accepter les cookies"


In [5]:
def try_accept_cookies(driver, timeout=5):
    """
    Tente de cliquer sur un bouton 'Accepter' / 'Accept' / 'OK' / 'Continuer'.
    Renvoie True si un bouton a été cliqué, False sinon.
    """
    start = time.time()
    xpaths = [
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accepter')]",
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accept')]",
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'ok')]",
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accepter tout')]",
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accept all')]",
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'continuer')]",
        "//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'agree')]",
        "//a[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accepter')]",
        "//a[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accept')]",
        "//input[@type='button' and (contains(translate(@value,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accepter') or contains(translate(@value,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'accept'))]"
    ]
    while time.time() - start < timeout:
        for xp in xpaths:
            try:
                elems = driver.find_elements(By.XPATH, xp)
                for el in elems:
                    if el.is_displayed() and el.is_enabled():
                        try:
                            el.click()
                            return True
                        except ElementClickInterceptedException:
                            driver.execute_script("arguments[0].click();", el)
                            return True
            except Exception:
                pass
        time.sleep(0.5)
    return False

Aller sur Bing News et faire la recherche

In [ ]:
driver.get("https://www.bing.com/news")
time.sleep(2)
try_accept_cookies(driver, timeout=3)  # Accepter les cookies sur Bing

search = driver.find_element(By.NAME, "q")
search.clear()
search.send_keys("Trump") # On met notre recherche dans la barre du moteur
search.submit()

# attendre les résultats
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.title, h2 a, h3 a"))
)

# Récupérer les 10 premiers articles
articles = []
for link in driver.find_elements(By.CSS_SELECTOR, "a.title, h2 a, h3 a"):
    title = link.text.strip()
    url = link.get_attribute("href")
    if title and url and url.startswith("http"):
        articles.append((title, url))
    if len(articles) >= 10:
        break

print(f"{len(articles)} articles trouvés.\n")

10 articles trouvés.



Scraper chaque article et accepter les cookies à chaque fois

In [7]:
data = []

for i, (title, url) in enumerate(articles, start=1):
    print(f"Article {i}/{len(articles)} : {title}")
    print(url)

    try:
        driver.get(url)
        time.sleep(3)

        # Accepter les cookies sur le nouveau site
        try_accept_cookies(driver, timeout=5)

        # Extraire les paragraphes d'article
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "article p, main p, p")
        text = " ".join([p.text for p in paragraphs if len(p.text) > 40])
        text = text.strip()[:4000]

        if not text:
            text = "(Aucun texte trouvé)"

        data.append({"titre": title, "url": url, "texte": text})
        print(f"{len(text)} caractères extraits.\n")

    except Exception as e:
        print("Erreur sur cet article : {e}\n")
        data.append({f"titre": title, "url": url, "texte": "(Erreur ou vide)"})


Article 1/10 : États-Unis : les démocrates victorieux des premiers scrutins du second mandat de Trump
https://www.france24.com/fr/vid%C3%A9o/20251105-%C3%A9tats-unis-les-d%C3%A9mocrates-victorieux-des-premiers-scrutins-du-second-mandat-de-trump
480 caractères extraits.

Article 2/10 : Donald Trump en condamné en mort : une statue à l’effigie du président américain fait sensation
https://www.ladepeche.fr/2025/11/05/donald-trump-sangle-sur-une-croix-une-statue-a-leffigie-du-president-americain-fait-sensation-13034241.php
2183 caractères extraits.

Article 3/10 : Ces dix moments stupéfiants depuis l’élection de Donald Trump, il y a un an
https://www.ouest-france.fr/monde/etats-unis/donald-trump/ces-dix-moments-stupefiants-depuis-lelection-de-donald-trump-il-y-a-un-an-8b5fe75c-b989-11f0-b227-510f72443e00
4000 caractères extraits.

Article 4/10 : Zohran Mamdani n’est pas le seul à avoir infligé un revers à Trump et aux républicains
https://www.msn.com/fr-fr/actualite/monde/zohran-mamdani-n-

Sauvegarder dans un CSV

In [9]:
driver.quit() # attention à bien quitter le browser (bug potentiel sinon)

In [10]:
csv_path = "articles_selenium.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["titre", "url", "texte"])
    writer.writeheader()
    writer.writerows(data)

print(f"Scraping terminé. {len(data)} articles enregistrés dans {csv_path}")

Scraping terminé. 10 articles enregistrés dans articles_selenium.csv
